In [2]:
# run(nba, model=SVC(random_state=0))
import datetime
import string
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from utils import *

# run(nba, model=SVC(random_state=0))
# Tuned SVC - rbf kernel
# model = SVC(kernel=kernels[0], random_state=0)
# run(nba, model=model)

In [7]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0).fit(X, y)

/home/jacob/.local/share/virtualenvs/mlflow_prac-rRmHmFzQ/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [3]:
conn = sql_connection('ml_models')
past_games = pd.read_sql_query('select * from ml_past_games', conn)
tonights_games = pd.read_sql_query('select * from ml_tonights_games', conn)

# completed_games = completed_games.to_numpy()
# tonights_games = tonights_games.to_numpy()

In [4]:
completed_ml = pd.read_sql_query('select * from tonights_games_ml', conn)


ProgrammingError: (psycopg2.errors.UndefinedTable) relation "tonights_games_ml" does not exist
LINE 1: select * from tonights_games_ml
                      ^

[SQL: select * from tonights_games_ml]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [31]:
past_games_full = pd.read_sql_query('select * from ml_past_games', conn)
tonights_games_full = pd.read_sql_query('select * from ml_tonights_games', conn).sort_values('home_team_avg_pts_scored')

In [32]:
tonights_games = tonights_games_full.drop(['home_team', 'away_team', 'proper_date', 'outcome'], axis = 1)

In [33]:
past_games = pd.read_sql_query('select * from ml_past_games', conn)
past_games_outcome = past_games['outcome']
past_games = past_games.drop(['home_team', 'away_team', 'proper_date', 'outcome'], axis = 1)

past_games_outcome = past_games_outcome.to_numpy()
past_games = past_games.to_numpy()

In [34]:
clf = LogisticRegression(random_state=0).fit(past_games, past_games_outcome)
clf.score(past_games, past_games_outcome)

0.6609625668449198

In [44]:
# feature importance
importance = clf.coef_[0]
print(importance)

[-0.04978847  0.04070776 -0.0367953   0.00377045  0.00076865  0.30607929
  0.06662241 -0.01126052  0.00899668 -0.0153359  -0.00437527 -0.46615876]


In [35]:
clf.predict(tonights_games)

/home/jacob/.local/share/virtualenvs/mlflow_prac-rRmHmFzQ/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


array([1, 1, 1, 0, 0, 0, 1])

In [51]:
clf.predict_proba(tonights_games)

/home/jacob/.local/share/virtualenvs/mlflow_prac-rRmHmFzQ/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


array([[0.48876101, 0.51123899],
       [0.2590111 , 0.7409889 ],
       [0.27315915, 0.72684085],
       [0.56128013, 0.43871987],
       [0.57157408, 0.42842592],
       [0.77698252, 0.22301748],
       [0.40668747, 0.59331253]])

In [68]:
tonights_ml = pd.DataFrame(clf.predict_proba(tonights_games)).rename(columns = {0: "away_team_predicted_win_pct", 1: "home_team_predicted_win_pct"})

tonights_games_final = tonights_games_full.reset_index().drop('outcome', axis = 1)

tonights_games_final['home_team_predicted_win_pct'] = tonights_ml['home_team_predicted_win_pct'].round(3)
tonights_games_final['away_team_predicted_win_pct'] = tonights_ml['away_team_predicted_win_pct'].round(3)

/home/jacob/.local/share/virtualenvs/mlflow_prac-rRmHmFzQ/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
